# Base Class

In [1]:
class Function():
    '''
    this is like a function that is responsible for getting its own inputs 
    this obejct is strictly serial. in a real actor it would have timeouts
    and parallel ability to communicate with many nodes at once.
    '''
    
    def __init__(self, inputs: dict = None):
        self.set_inputs(inputs=inputs or {})
        self.clear(memory=True)
    
    def set_inputs(self, inputs: dict):
        self.inputs = inputs
        self.kwargs = {name: None for name in self.inputs.keys()}

    def clear(self, memory: bool = False):
        self.cached = False
        self.getout = False
        if memory:
            self.outputs = None
            self.latest = None

    def run(self, gas: int = 1, verbose: bool = False):
        '''
        gas can specify if we should pull from cache or not in this way:
        
        0 - return cache if you have it, otherwise get your inputs and do your function
        1 - ask inputs for cached and run your own functionality (default)
        2 - when you ask your inputs for data, make sure they recompute instead of cache
        3 - when your inputs ask for their inputs, make sure they recompute...
        4+ - so on and so forth...
        -1 - infinite gas no matter what. (will lead to an infinite loop in a non-dag structure)
        

        There's no mechanism to force a DAG structure so force_refresh_all and force_refresh_while are
        not suggested practice as it can lead to infinite loops
        '''
        if gas == 0 and self.cached:
            pass
        elif gas == 0 and not self.cached:
            self.aquire(gas=0)
            self.output = self.function()
            self.cached = True
        elif gas >= 1:
            self.aquire(gas=gas - 1)
            self.output = self.function()
            self.cached = True
        elif gas == -1:
            self.aquire(gas)
            self.output = self.function()
            self.cached = True
        else:
            self.aquire(0)
        return self.get()

    def aquire(self, gas: int = 0):
        for name, function_object in self.inputs.items():
            self.kwargs[name] = function_object.run(gas)
        
    def get(self):
        return self.output

    def function(self):
        ''' main '''
        return self.output

# Examples

In [ ]:
class A(Function):
    
    def __init__(self, inputs: dict = None):
        super(A, self).__init__(inputs)
        
    def function(self):
        print('A running!')
        return 1

In [ ]:
a = A()

In [ ]:
a.inputs

In [ ]:
a.cached

In [ ]:
print(a.output)

In [ ]:
a.run()

In [ ]:
print(a.output)

In [ ]:
class B(Function):
    
    def __init__(self, inputs: dict = None):
        super(B, self).__init__(inputs)
        
    def function(self):
        print('B running!')
        return 2

In [ ]:
b = B()

In [ ]:
# b.run()

In [ ]:
print(b.output)

In [ ]:
print(b.cached)

In [ ]:
class C(Function):
    
    def __init__(self, inputs: dict = None):
        super(C, self).__init__(inputs)
        
    def function(self):
        print('C running!')
        return self.kwargs['A'] + self.kwargs['B']

In [ ]:
c = C({'A': a, 'B': b})

In [ ]:
c.kwargs

In [ ]:
c.aquire()

In [ ]:
c.kwargs

In [ ]:
c.cached

In [ ]:
c.run()

In [ ]:
class D(Function):
    
    def __init__(self, inputs: dict = None):
        super(D, self).__init__(inputs)
        
    def function(self):
        return self.transformation(**self.kwargs)
    
    def transformation(self, **kw):
        print('D running!')
        return kw['M']+1

In [ ]:
d = D({'M': c})

In [ ]:
d.run()

In [ ]:
d.run(0)

In [ ]:
d.run(1)

In [ ]:
d.run(2)

In [ ]:
d.run(3)

In [ ]:
d.run(-1)

# recursion

The following structure is a hash structure it contains one and only one loop. We can't use `gas=-1` because it will loop forever. But we can use `gas=-2` as long as we give something in the loop a default initial value. This works because we know they converge.

In [2]:
class X(Function):
    
    def __init__(self, inputs: dict = None):
        super(X, self).__init__(inputs)
        self.name = 'x'
        
    def function(self):
        return self.transformation(**self.kwargs)
    
    def transformation(self, **kw):
        if kw['Y'] < 10:
            i = kw['Y']+1
        else: 
            i = kw['Y']
        print('X running! returning', i)
        return i
    
    
class Y(Function):
    
    def __init__(self, inputs: dict = None):
        super(Y, self).__init__(inputs)
        self.name = 'y'
        
    def function(self):
        return self.transformation(**self.kwargs)
    
    def transformation(self, **kw):
        if kw['X'] < 6:
            i = kw['X']+1
        else: 
            i = kw['X']
        print('Y running! returning', i)
        return i
    
    
class Z(Function):
    
    def __init__(self, inputs: dict = None):
        super(Z, self).__init__(inputs)
        self.name = 'z'
        
    def function(self):
        return self.transformation(**self.kwargs)
    
    def transformation(self, **kw):
        i = kw['Y']+1
        print('Z running! returning', i)
        return i

In [3]:
x = X()
y = Y({'X': x})
z = Z({'Y': y})
x.set_inputs({'Y': y})

In [4]:
x.kwargs

{'Y': None}

In [5]:
# if something doesn't have a 'default' cached value, the loop with be infinite.
x.output = 0
x.cached = True

In [6]:
z.run(20)

Y running! returning 1
X running! returning 2
Y running! returning 3
X running! returning 4
Y running! returning 5
X running! returning 6
Y running! returning 6
X running! returning 7
Y running! returning 7
X running! returning 8
Y running! returning 8
X running! returning 9
Y running! returning 9
X running! returning 10
Y running! returning 10
X running! returning 10
Y running! returning 10
X running! returning 10
Y running! returning 10
Z running! returning 11


11

# Visualization

In theory we could build a dask dag graph from the inputs values of the objects. but this is hard to test and develop since this machine can't install conda, and we need conda to properly install graphviz because we're using dask to visiualize it...

In [ ]:
1/0

In [ ]:
import dask
dag = {
    'A': (a.run, *a.inputs.keys()),
    'B': (b.run, *b.inputs.keys()),
    'C': (c.run, *c.inputs.keys()),
    'D': (d.run, *d.inputs.keys()),
}
dask.visualize(dag)